# 4-7. 프로젝트: 멋진 작사가 만들기

**Step 1. 데이터 다운로드**

이미 `실습(1) 데이터 다듬기`에서 `Cloud shell`에 심볼릭 링크로 `~/aiffel/lyricist/data`를 생성하셨다면, `~/aiffel/lyricist/data/lyrics`에 데이터가 있습니다.

**Step 2. 데이터 읽어오기**

`glob` 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. `glob` 를 활용하여 모든 txt 파일을 읽어온 후, `raw_corpus` 리스트에 문장 단위로 저장하도록 할게요!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import glob
import os

txt_file_path = '/content/drive/MyDrive/아이펠/풀잎스쿨/data/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담는다.
# raw_corpus에는 문장 단위로 데이터가 들어간다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기(문장 갯수):", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기(문장 갯수): 187088
Examples:
 ['Hey, Vietnam, Vietnam, Vietnam, Vietnam', 'Vietnam, Vietnam, Vietnam Yesterday I got a letter from my friend', 'Fighting in Vietnam']


**Step 3. 데이터 정제**

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

`preprocess_sentence()` 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 **토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기** 를 권합니다.

In [3]:
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1 # strip()은 문자열 양쪽 끝에 있는 공백을 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 # \1은 첫번째 그룹을 뜻한다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5 
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
# corpus 정제된 문장을 담을 리스트
corpus = []

# preprocess_sentence 함수를 이용해서 정제된 문장을 담아준다.
# 토큰의 개수가 15개를 넘어가는 문장을 제외
# split 합수로 공백을 기준으로 단어를 나누어 준다.
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if len(sentence.split()) >= 13:  # 토큰 15개 이상이면 삭제하라고 되어있는데, 그 뒤에 학습데이터수가 124960개 이상이면 안된다고 해서 다시 줄임
        continue
    corpus.append(preprocess_sentence(sentence))
        
# 정제된 결과를 10개 확인
corpus[:10]

['<start> hey , vietnam , vietnam , vietnam , vietnam <end>',
 '<start> vietnam , vietnam , vietnam yesterday i got a letter from my friend <end>',
 '<start> fighting in vietnam <end>',
 '<start> and this is what he had to say <end>',
 '<start> tell all my friends that i ll be coming home soon <end>',
 '<start> my time it ll be up some time in june <end>',
 '<start> don t forget , he said to tell my sweet mary <end>',
 '<start> her golden lips as sweet as cherries and it came from <end>',
 '<start> vietnam , vietnam , vietnam , vietnam <end>',
 '<start> it was addressed from vietnam <end>']

In [5]:
# 토큰화

import tensorflow as tf
import numpy as np

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다 (전체 단어의 수 12000개로 제한)
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )

    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # fit_on_texts는 리스트 형태로 결과를 반환
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
 
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

print('\n-------------------------\n')
print('텐서의 갯수:', len(tensor))

[[   2  141    4 ...    0    0    0]
 [   2 1296    4 ...    0    0    0]
 [   2 1072   14 ...    0    0    0]
 ...
 [   2   44    6 ...    0    0    0]
 [   2   31    7 ...    0    0    0]
 [   2  304    1 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f9987539c10>

-------------------------

텐서의 갯수: 158876


**Step 4. 평가 데이터셋 분리**

훈련 데이터와 평가 데이터를 분리하세요!

`tokenize()` 함수로 데이터를 Tensor로 변환한 후, `sklearn` 모듈의 `train_test_split()` 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. `단어장의 크기는 12,000 이상` 으로 설정하세요! **총 데이터의 20%** 를 평가 데이터셋으로 사용해 주세요!

In [6]:
from sklearn.model_selection import train_test_split

src_input = tensor[:, :-1]  # 행 전체, 마지막 열을 제외한 모든 열
tgt_input = tensor[:, 1:]   # 행 전체, 첫번째 열을 제외한 모든 열 (<start>제외)

enc_train, enc_val, dec_train, dec_val = train_test_split(
    src_input, tgt_input,
    random_state=2022,
    test_size = 0.2
)

여기까지 올바르게 진행했을 경우, 아래 실행 결과를 확인할 수 있습니다.

In [7]:
# encoding, decoding의 약자인듯. 갑자기 변수명이 바뀌어서 헷갈렸다.
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127100, 32)
Target Train: (127100, 32)


In [8]:
# shuffle에서 쓰인다.
# shuffle 함수는 고정된 버퍼 크기로 데이터를 섞는데, 
# 완전히 랜덤하게 섞기 위해서는 입력된 데이터 크기보다 큰 수를 입력해 주어야 한다.
# 지금은 train 데이터셋 전체 갯수(127100)를 buffer_size로 설정했다.
BUFFER_SIZE = len(enc_train)
# batch_size : 한 번에 읽어올 데이터의 갯수
BATCH_SIZE = 256
# 여기서 정한 steps_per_epoch은 뒤에 사용되지 않았다.
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
# 앞에서 num_words = 12000으로 정해주었음
VOCAB_SIZE = tokenizer.num_words + 1

# tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환
# Premade Estimator를 사용하기 위해서는 feature 데이터와 label 데이터가 함께 전달하여 dataset을 생성해야 한다.
train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
# drop_remainder=True는 마지막 남은 데이터를 버린다는 옵션 설정
# 127100 // 256 : 나머지 데이터는 버림
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder = True)
train_dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

In [9]:
# validation 데이터셋도 텐서를 tf.data.Dataset 객체로 변환시켜준다.
test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder = True)
test_dataset

<BatchDataset shapes: ((256, 32), (256, 32)), types: (tf.int32, tf.int32)>

만약 결과가 다르다면 천천히 과정을 다시 살펴 동일한 결과를 얻도록 하세요! 만약 학습 데이터 개수가 124960보다 크다면 위 Step 3.의 데이터 정제 과정을 다시 한번 검토해 보시기를 권합니다.

**Step 5. 인공지능 만들기**

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # embedding_size : 워드벡터의 차원 수
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
for src_sample, tgt_sample in train_dataset.take(1):
    break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 32, 12001), dtype=float32, numpy=
array([[[ 3.30475654e-04,  1.44331323e-04, -1.80458257e-04, ...,
          5.72627650e-05,  1.89925195e-04, -5.20303838e-05],
        [ 5.45686751e-04,  2.77402432e-04, -2.06524841e-04, ...,
          1.51327302e-04,  2.20560847e-04,  1.64475623e-05],
        [ 5.07978722e-04,  3.62450606e-04, -1.52246706e-04, ...,
          2.24055548e-04,  2.74231279e-04,  1.81390365e-04],
        ...,
        [-3.81102413e-03, -1.57169509e-03, -2.70972052e-03, ...,
         -2.07704306e-03,  4.47048945e-03, -3.54059716e-03],
        [-3.82430037e-03, -1.56424055e-03, -2.72766664e-03, ...,
         -2.12893402e-03,  4.51802835e-03, -3.54226422e-03],
        [-3.83470999e-03, -1.55552232e-03, -2.74355640e-03, ...,
         -2.17411807e-03,  4.55840491e-03, -3.54350242e-03]],

       [[ 3.30475654e-04,  1.44331323e-04, -1.80458257e-04, ...,
          5.72627650e-05,  1.89925195e-04, -5.20303838e-05],
        [ 5.80939348e-04,  5.84387853e-05, -2

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
# optimizer는 loss를 최소화하는 w(가중치)를 찾는 것 (cs231n에 나온 내용)
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

# model.compile(loss, optimizer, metrics)
model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, validation_data = test_dataset, epochs=10)

Epoch 1/10
496/496 [==============================] - 214s 426ms/step - loss: 1.6324 - val_loss: 1.5506
Epoch 2/10
496/496 [==============================] - 221s 446ms/step - loss: 1.3928 - val_loss: 1.3495
Epoch 3/10
496/496 [==============================] - 221s 445ms/step - loss: 1.3070 - val_loss: 1.3055
Epoch 4/10
496/496 [==============================] - 221s 445ms/step - loss: 1.2597 - val_loss: 1.2718
Epoch 5/10
496/496 [==============================] - 221s 446ms/step - loss: 1.2182 - val_loss: 1.2440
Epoch 6/10
496/496 [==============================] - 221s 446ms/step - loss: 1.1803 - val_loss: 1.2206
Epoch 7/10
496/496 [==============================] - 221s 445ms/step - loss: 1.1449 - val_loss: 1.2012
Epoch 8/10
496/496 [==============================] - 221s 445ms/step - loss: 1.1111 - val_loss: 1.1832
Epoch 9/10
496/496 [==============================] - 221s 446ms/step - loss: 1.0785 - val_loss: 1.1674
Epoch 10/10
496/496 [==============================] - 222s 446m

데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다.
    while True:
        predict = model(test_tensor) # 입력받은 문장의 텐서를 입력합니다.
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다.
        
        # 우리 모델이 새롭계 예측한 단어를 입력 문장의 뒤에 붙여줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면 while 루프를 돌면서 다음 단어를 예측해야합니다.
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break
    
    generated = ""
    
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다.
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [15]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m a survivor <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> if you", max_len=20)

'<start> if you re a little girl , you know it <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i wonder", max_len=20)


'<start> i wonder why you know <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i wanted", max_len=20)

'<start> i wanted to be a little selfish <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> she loves", max_len=20)

'<start> she loves me <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> please give ", max_len=20)

'<start> please give me a million reasons <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> if you want ", max_len=20)

'<start> if you want to be your lover <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> say ", max_len=20)

'<start> say it s time <end> '

> Q4. 모델이 생성한 가사 한 줄을 제출하세요.<br><br>
          i wonder why you know

# 회고

- ***이번 프로젝트에서 어려웠던 점,***<br>

    `def tokenize(corpus)` 함수의 코드에서 tensor객체가 아닐때 부터 tensor라고 변수명이 지정되어있어서 많이 헷갈렸다. 그리고 validation을 왜 `train_test_split`으로 나누었는지 이해가 되지 않았다... 어떻게 잘 마무리했지만 헷갈릴 법한 소지가 너무 많지 않았나 생각한다.
    `model.fit(validation_split=0.2)` 이런식으로 기본 코드가 짜여있었더라면 좋았을 것 같다. 코드가 너무 길고 아직은 내가 파이썬에 능숙하진 못해서 코드를 변경시키려고 했으나 엄두가 나지 않아서 그렇게 하진 못했다.

    그리고 지문중에 토큰 15개 이상의 문장을 삭제하라는 말이 있었는데, 15개 이상 문장을 삭제했을때, train 데이터셋의 갯수가 124960개를 넘어서 결국 13개 이상 문장을 삭제했어야 했다. 아이펠에서 점검이 필요한 사항이라고 생각한다.

- ***프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.***

    토큰 15개 이상인 문장을 삭제하라고 해서, `def tokenize()`안에서 문장을 토큰화 한 다음에 만들어진 텐서를 아래와 같이 삭제를 시도하였다.

In [23]:
for ten in tensor:
    if len(ten) >= 15:
        del ten

하지만, 텐서는 이렇게 `del`로 삭제가 불가능하였고, ndarray로 변환시켜서 삭제한 뒤 다시 텐서로 변형을 해 주어야한다고 블로그 검색을 통해 알게되었다. 하지만, 텐서를 ndarray로 변형시키는 함수인 `tensor.numpy()`와 `Tensor.eval()`, `Tensorflow.session()` 이 세 개를 다 적용해 보았으나 제대로 변형이 되질 않아서 계속 모두 실패하였다.

오기가 생겨서 6시간동안 도전해 보았으나 실패... 
이후에 다시 보니, 패딩을 넣기 전엔 `tokenizer.fit_on_texts(corpus)`에서 리스트를 반환했기 때문에, ndarray도 아니었고, tensor도 아니었다. 그래서 `tensor.numpy()`코드가 자꾸 에러가 났던 것이었다.
그 코드로 다시 또 시도해 보았으나, 이번엔 `del`에서 오류가 났다. 리스트의 요소를 삭제할 수는 있지만, 리스트 자체를 삭제하는 것이 아니기 때문이었다... (그런데 `del List명`이면 리스트 자체를 삭제 가능하다고 해서 조금 헷갈린다. for문에서 주어진 것이 바로 할당된 값이 아니어서 그런것인지..?)

결국 원래 있던 코드인, `sentence.split()`를 통해 토큰화 하기 전 문장을 공백으로 만들어 주는 함수를 사용해서 해결하였다. 아직도 토큰화를 하고 나서 토큰의 갯수로 할 수 있다면 좋았을거라는 아쉬움이 든다.

- **루브릭 평가 지표를 맞추기 위해 시도한 것들**.

>1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?<br>
        동작됨
    
>2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?<br>
        특수문자 제거, 토크나이저 생성, 패딩처리 모두 완료되었다.
    
> 3. 텍스트 생성모델이 안정적으로 학습되었는가?<br>
        학습됨

- ***만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정.***

    

- ***자기 다짐***

    파이썬 기본적인 문법에 대해서 좀 더 공부를 해야겠다는 생각이 들었다. 기본 문법에서 헷갈리니,코드 이해하는데 많은 시간이 소요되었다..